# Introduction to the `Keras` Functional API

A sequential model is an abstraction of a neural network model in which a linear stack of layers belongs to a single network and processes a single input to produce a single output.

More complex models may have
- **multimodal inputs**: a model may have to process multiple independent inputs separately through different neural layers. The outputs of these modules are then merged using a merging module.
- **multiple outputs**: instead of training separate neural networks to predict separate outputs, we can let the neural networks learn from backpropagation through both outputs using corrrelations between them.
- **internal branching between layers**: see ResNet, Inception.

Not possible to implement such complex models using the `Sequential` `keras` approach. Have to use a more general approach called the **functional API**, in which each layer is a function to which one or more input tensors can be passed in order to produce one or more output tensors. 

## Overview

In the functional API, each layer in the neural network acts as a function which accepts input tensors and returns output tensors. We manipulate tensors directly by instantiating them, passing them as inputs to layers acting as functions, accepting output tensors of these layers, and passing them to other layers, and finally creating a `Model` that links our `Input` tensor with our `Output` tensor.

In [48]:
from tensorflow.keras import Input, layers

In [49]:
# Instantiating an Input tensor
input_tensors = Input(shape=(32, ))

In [50]:
# Creating a Dense layer that will act as a function to process this input tensor
dense = layers.Dense(units=32, activation='relu')

In [51]:
# Output tensor will be the result of passing the input tensor to the layer (function)
output_tensor = dense(input_tensor)

## Comparing Sequential and Functional Models

### `Sequential` Model 

In [62]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras import layers 
from tensorflow.keras import Input

In [63]:
# Instantiate a Sequential object
seq_model = Sequential()

In [64]:
# This model is just a sequential stack of layers that transform some input
seq_model.add(layers.Dense(units=32, activation='relu', input_shape=(64, )))
seq_model.add(layers.Dense(units=32, activation='relu'))
seq_model.add(layers.Dense(units=10, activation='softmax'))

In [65]:
# Summary of the model
seq_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_24 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_25 (Dense)             (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


### `Functional` Model

In [66]:
# Instantiate an Input tensor of a specific shape/dimension
input_tensor = Input(shape=(64, ))     # Each sample is a 64-dimensional vector

In [67]:
# Create Dense input layer as a function, call on the input, and store output tensor
x = layers.Dense(units=32, activation='relu')(input_tensor)

In [68]:
# Second layer will be called on the output of the first
x = layers.Dense(units=32, activation='relu')(x)

In [69]:
# Final output layer will be called on the output of the last layer, and will return output_tensor
output_tensor = layers.Dense(units=10, activation='softmax')(x)

In [70]:
# Finally, instantiate a Model object using the IO tensors
func_model = Model(input_tensor, output_tensor)

In [71]:
# Identical to this sequential model
func_model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 64)]              0         
_________________________________________________________________
dense_26 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_27 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_28 (Dense)             (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


`output_tensor` was obtained by repeatedly transforming an `input_tensor` through consecutive `layers`. The only reason we can make `Model` object using these tensors is because under the hood, `keras` will collect and combine all layers used to transform an `input_tensor` into an `output_tensor` into a single graph-like structure.

This approach will not work if we try to link the `output_tensor` with an `unrelated_input_tensor`, because the two will not have been linked with a `layer` transformations.

In [72]:
unrelated_input = Input(shape=(32, ))

In [73]:
bad_model = model = Model(unrelated_input, output_tensor)

ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_4:0", shape=(None, 64), dtype=float32) at layer "input_4". The following previous layers were accessed without issue: []

### Compiling and Training 
The API for the functional model is the same as that of the `Sequential` models we have built so far.

In [77]:
# Same call to compile
func_model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [78]:
import numpy as np

# Generate dummy data to train model on
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

In [79]:
# Same call to fit
func_model.fit(x_train, y_train, batch_size=128, epochs=10)

Train on 1000 samples
Epoch 1/10
1000/1000 [==============================] - 0s 416us/sample - loss: 11.6512
Epoch 2/10
1000/1000 [==============================] - 0s 31us/sample - loss: 11.8705
Epoch 3/10
1000/1000 [==============================] - 0s 58us/sample - loss: 12.5911
Epoch 4/10
1000/1000 [==============================] - 0s 37us/sample - loss: 13.7848
Epoch 5/10
1000/1000 [==============================] - 0s 38us/sample - loss: 15.4912
Epoch 6/10
1000/1000 [==============================] - 0s 46us/sample - loss: 17.5808
Epoch 7/10
1000/1000 [==============================] - 0s 28us/sample - loss: 19.9762
Epoch 8/10
1000/1000 [==============================] - 0s 28us/sample - loss: 22.6754
Epoch 9/10
1000/1000 [==============================] - 0s 28us/sample - loss: 25.2862
Epoch 10/10
1000/1000 [==============================] - 0s 32us/sample - loss: 27.8254


In [81]:
# Still same call to scoring function
score = func_model.evaluate(x_train, y_train)

1000/1000 [==============================] - 0s 125us/sample - loss: 29.5689
